In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [7]:
# URL of the website
URL = "https://www.redbus.in/online-booking/jksrtc"
# Set up the Chrome driver
driver = webdriver.Chrome()
driver.get(URL)
driver.maximize_window()
time.sleep(5)    # Wait for the page to load

In [8]:
# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name[i].text,
                    "Bus_Type": bus_type[i].text,
                    "Departing_Time": departing_time[i].text,
                    "Duration": duration[i].text,
                    "Reaching_Time": reaching_time[i].text,
                    "Star_Rating": star_rating[i].text if i < len(star_rating) else '0',
                    "Price": price[i].text,
                    "Seat_Availability": seat_availability[i].text if i < len(seat_availability) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)


        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('jammu.csv', index=False)

 #Close the driver
driver.quit()

Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-srinagar: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF61DEB9632+30946]
	(No symbol) [0x00007FF61DE6E3C9]
	(No symbol) [0x00007FF61DD66FDA]
	(No symbol) [0x00007FF61DDB822C]
	(No symbol) [0x00007FF61DDB850C]
	(No symbol) [0x00007FF61DDFDCB7]
	(No symbol) [0x00007FF61DDDCAAF]
	(No symbol) [0x00007FF61DDFB041]
	(No symbol) [0x00007FF61DDDC813]
	(No symbol) [0x00007FF61DDAA6E5]
	(No symbol) [0x00007FF61DDAB021]
	GetHandleVerifier [0x00007FF61DFEF83D+1301229]
	GetHandleVerifier [0x00007FF61DFFBDB7+1351783]
	GetHandleVerifier [0x00007FF61DFF2A03+1313971]
	GetHandleVerifier [0x00007FF61DEEDD06+245686]
	(No symbol) [0x00007FF61DE7758F]
	(No symbol) [0x00007FF61DE73804]
	(No symbol) [0x00007FF61DE73992]
	(No symbol) [0x00007FF61DE6A3EF]
	BaseThreadInitThunk [0x00007FFF46E8257D+29]
	RtlUserThreadStart [0x00007FFF47CAAF28+40]

Error occurred while scraping bus details for https://www.redbus.in/bus-tick

In [9]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Jammu (j and k) to Srinagar,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,1.4,719,36 Seats available
1,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:50,12h 40m,09:30,4.5,INR 610,15 Seats available
2,Jammu (j and k) to Haridwar,https://www.redbus.in/bus-tickets/jammu-to-har...,,,,,,,,
3,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi,Gola Bus Service,VE A/C Seater / Sleeper (2+1),20:20,11h 30m,07:50,4.8,INR 399,12 Seats available
4,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi,Zingbus Plus,,20:10,13h 10m,09:20,4.6,382,
5,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi,,,,,,,,
6,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi,,,,,,,,
7,Jammu (j and k) to Delhi,https://www.redbus.in/bus-tickets/jammu-to-delhi,,,,,,,,
8,Delhi to Jammu (j and k),https://www.redbus.in/bus-tickets/delhi-to-jammu,Laxmi holidays,Volvo 9600 Multi-Axle A/C Sleeper (2+1),21:30,12h 45m,10:15,4.5,INR 1000,17 Seats available
9,Delhi to Jammu (j and k),https://www.redbus.in/bus-tickets/delhi-to-jammu,,,,,,,,
